In [1]:
import numpy as np 
import pandas as pd
import regex as re
import unicodedata as ud
import nltk
from nltk.stem.isri import ISRIStemmer
from nltk.tokenize import wordpunct_tokenize
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.model_selection import train_test_split
import multiprocessing
from tqdm import tqdm
from sklearn import utils
from sklearn.metrics import accuracy_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
import numpy as np
import arabic_reshaper as ar
from bidi.algorithm import get_display
import matplotlib.pyplot as plt

Movies Dataset


EDA functions

Cleansing functions

We reduce the number of labels by grouping similar genres into one label,

    ﺩﺭاﻣﺎ/ﺗﺸﻮﻳﻖ ﻭﺇﺛﺎﺭﺓ/ﺣﺮﻛﺔ/ﺟﺮﻳﻤﺔ/ﻣﻐﺎﻣﺮاﺕ/ﻏﻤﻮﺽ = دراما
    ﻋﺎﺋﻠﻲ/ ﻛﻮﻣﻴﺪﻱ = ﻛﻮﻣﻴﺪﻱ

While the rest of the labels remain unchanged.


Short movies cannot be predicted contextually, therefore we assign the genre "ﻗﺼﻴﺮ" to movies with duration less than 45 minutes, and remove them from our yet to be trained dataset.

Compute TF

In [144]:
#word set contains all unique words through all movies
def unique_words(syn):
    word_set=[]
    for i in range(len(syn)):
        for j in range(len(syn[i])):
            if syn[i][j] not in word_set:
                word_set.append(syn[i][j])
    return  word_set          

In [145]:
word_set=unique_words(syn)

In [146]:
wordDict=dict.fromkeys(word_set,0)
(wordDict)

{'تدور': 0,
 'احداث': 0,
 'الفيلم': 0,
 'خيريه': 0,
 'الفتاه': 0,
 'الثريه': 0,
 'مخطوبه': 0,
 'لأحد': 0,
 'ابناء': 0,
 'عمومتها': 0,
 'وبعد': 0,
 'تفقد': 0,
 'اسرتها': 0,
 'الثروه': 0,
 'تملكها': 0,
 'يتخلى': 0,
 'ابن': 0,
 'العم': 0,
 'يقترن': 0,
 'بفتاه': 0,
 'وخلال': 0,
 'بدايه': 0,
 'الأسره': 0,
 'جديد': 0,
 'تقف': 0,
 'قدميها': 0,
 'تتعرف': 0,
 'فايز': 0,
 'وترتبط': 0,
 'اعتراض': 0,
 'العائله': 0,
 'اصوله': 0,
 'الاجتماعيه': 0,
 'المتواضعه': 0,
 'كونه': 0,
 'استطاع': 0,
 'تكوين': 0,
 'ثروه': 0,
 'يعتد': 0,
 'يسافر': 0,
 'الأسطى': 0,
 'عبدالرحمن': 0,
 'صاحب': 0,
 'ورشه': 0,
 'الميكانيكا': 0,
 'زملاء': 0,
 'رحله': 0,
 'صيد': 0,
 'الصحراء': 0,
 'تاركًا': 0,
 'زوجته': 0,
 'زينب': 0,
 'وابنه': 0,
 'مصطفى': 0,
 'وأمه': 0,
 'وبينما': 0,
 'طريقهم': 0,
 'تهب': 0,
 'عاصفه': 0,
 'يختفى': 0,
 'بعدها': 0,
 'فيعتقد': 0,
 'الجميع': 0,
 'فيهم': 0,
 'مات': 0,
 'فيتولى': 0,
 'حسن': 0,
 'شقيق': 0,
 'امور': 0,
 'الورشه': 0,
 'يستطيع': 0,
 'إدارتها': 0,
 'ويتوقف': 0,
 'العمل': 0,
 'وتصبح': 0,
 'مديون

Compute IDf

Compute TF-IDF

Word2vec for synopses

In [158]:
from gensim.models import Word2Vec
import warnings
warnings.filterwarnings('ignore')
mymodel = Word2Vec(syn, min_count=1)
print(mymodel)

Word2Vec(vocab=30353, vector_size=100, alpha=0.025)


In [159]:
words = list(mymodel.wv.key_to_index )
words

['الزواج',
 'احمد',
 'زوجته',
 'ابنه',
 'زوجها',
 'الفتاه',
 'حب',
 'ليلى',
 'الشاب',
 'حسن',
 'يوجد',
 'فتاه',
 'يحاول',
 'ملخص',
 'يعمل',
 'الفيلم',
 'والدها',
 'الأحداث',
 'عادل',
 'ابن',
 'محمود',
 'تدور',
 'الحب',
 'يقوم',
 'العصابه',
 'شاب',
 'الأب',
 'ابنته',
 'حياه',
 'احداث',
 'بينهما',
 'رجل',
 'يعرف',
 'ناديه',
 'علاقه',
 'السجن',
 'حسين',
 'والده',
 'وبعد',
 'العمل',
 'تعمل',
 'المنزل',
 'يكتشف',
 'يقرر',
 'الراقصه',
 'البوليس',
 'صديقه',
 'الزوج',
 'يعيش',
 'الحياه',
 'القاهره',
 'لكنها',
 'يعود',
 'قصة',
 'وفى',
 'يقع',
 'محمد',
 'الزوجه',
 'الثرى',
 'اثناء',
 'الشرطه',
 'كمال',
 'بأن',
 'يتعرف',
 'زوجه',
 'يتم',
 'تقع',
 'وعندما',
 'القبض',
 'تعيش',
 'الأم',
 'يتزوج',
 'الدكتور',
 'عبد',
 'السينما',
 'يرفض',
 'مرة',
 'حياته',
 'المال',
 'قتل',
 'ويحاول',
 'والد',
 'حمدى',
 'صاحب',
 'تحاول',
 'يجد',
 'الأمر',
 'امها',
 'منى',
 'وحيد',
 'والتى',
 'تحب',
 'طريق',
 'هدى',
 'إليها',
 'سعاد',
 'سمير',
 'رغم',
 'تقوم',
 'الطبيب',
 'شريف',
 'إليه',
 'يحبها',
 'يطلب',
 'ولكنه',
 

In [160]:
print(mymodel.wv['ليلى'])

[-0.16289562  0.80188847  0.2773088   0.39047846  0.4723087  -0.6438106
  0.85193455  1.577934   -0.28320947 -0.19458914 -0.38434592 -1.1843038
 -0.41740552  0.41421735 -0.0415065  -0.6591845  -0.06188259 -0.7361667
 -0.07792824 -0.9290658   0.17376915  0.5104078   0.5319023  -0.2753525
 -0.2521466  -0.5575187  -0.22119918 -0.11123339 -0.91601044  0.20243746
  0.5408533  -0.1666307  -0.50692654 -0.66360754 -0.69239074  0.6641735
  0.4442723  -0.8617769  -0.3695945  -1.2367672   0.5991457  -0.33203593
 -0.40803114 -0.11787232  0.9611355   0.08499626 -0.37750703  0.05393294
  0.17508605  0.47025976  0.2933645  -0.9488622   0.07553674 -0.56258065
 -0.7099064   0.15122698  0.17400849  0.08893885 -0.7787798  -0.20200087
  0.4372287   0.06509756 -0.35536003 -0.30895764 -0.4992911   0.45622817
  0.01976602  0.45964426 -1.0433344   0.6973852  -0.5396926   0.33331832
  1.2697071  -0.70098305  0.06471214 -0.1922926  -0.0100771  -0.01937229
 -0.76932377  0.04335164 -0.44562584 -0.339784   -0.9992

In [161]:
mymodel.wv.most_similar('احمد',topn=10)

[('يقوم', 0.9998676180839539),
 ('وبعد', 0.9997941851615906),
 ('وفى', 0.9997811913490295),
 ('صديقه', 0.9997808933258057),
 ('الفتاه', 0.9997805953025818),
 ('عادل', 0.999778687953949),
 ('زوجه', 0.9997717142105103),
 ('اثناء', 0.9997628927230835),
 ('محمد', 0.9997608661651611),
 ('والدها', 0.999752938747406)]

# Questions to ASK

In [197]:

    actors_40s=[]
    actors_50s=[]
    actors_60s=[]
    actors_70s=[]
    for i in range(len(df)):
        actors=(df.at[i,'تمثيل']).split(',')
        if df.at[i,'تاريخ العرض']>=1940 and df.at[i,'تاريخ العرض']<=1949:
            for j in range(len(actors)):
                actors_40s.append(actors[j])
        if df.at[i,'تاريخ العرض']>=1950 and df.at[i,'تاريخ العرض']<=1959:
            for j in range(len(actors)):
                actors_50s.append(actors[j])
        if df.at[i,'تاريخ العرض']>=1960 and df.at[i,'تاريخ العرض']<=1969:
            for j in range(len(actors)):
                actors_60s.append(actors[j])
        if df.at[i,'تاريخ العرض']>=1970 and df.at[i,'تاريخ العرض']<=1979:
            for j in range(len(actors)):
                actors_70s.append(actors[j])
    actors_40s,actors_50s,actors_60s,actors_70s

(['سليمان نجيب',
  'أمينة رزق',
  'دولت أبيض',
  'عقيلة راتب',
  'سلوى علام',
  'زوزو نبيل',
  'نادية أحمد',
  'نينا زاكى',
  'أنور وجدي',
  'حسن فايق',
  'فاخر فاخر',
  'محمود المليجي',
  'زكي إبراهيم',
  'حسن مختار صقر',
  'عزيزة محمد',
  'مرجريت صفير',
  'مختار عثمان',
  'لطفي الحكيم',
  'عزيزة أمير',
  'محمود ذو الفقار',
  'أنور وجدي',
  'نجمة إبراهيم',
  'عبدالسلام النابلسي',
  'ثريا فخري',
  'عمر وصفى',
  'سرينا إبراهيم',
  'حسن مختار صقر',
  'ممدوح ذو الفقار',
  'إستيفان روستي',
  'عز الدين ذو الفقار',
  'أحمد علام',
  'زوزو شكيب',
  'عباس فارس',
  'محمد الديب',
  'زينات صدقي',
  'علي رشدي',
  'فتحي الصافوري',
  'بديعة صادق',
  'ميمي شكيب',
  'محسن سرحان',
  'روحية خالد',
  'فردوس محمد',
  'علية الحبشية',
  'ثريا حلمي',
  'ظريفة',
  'الشيخة عزيزة',
  'فؤاد شفيق',
  'منسى فهمي',
  'أنور وجدي',
  'عبدالفتاح القصري',
  'عبدالمجيد شكري',
  'يحيى نجاتي',
  'مصطفى الجزار',
  'محمد ابراهيم',
  'عباس بدوى',
  'فؤاد فهيم',
  'إدمون تويما',
  'محمد الكحلاوي',
  'عبدة السروجي',
  'حسن بيوم

What is the average Duration for films per Decade?

What is the average duration for films for each genre?

What is the numberof films per genre in 1940s ?

What is the number of films per genre in 1950s ?

What is the numberof films per genre in 1960s ?

What is the numberof films per genre in 1970s ?

what is the most 5 actors appearing in 1940s ?

what is the most 5 actors appearing in 1950s ?

what is the most 5 actors appearing in 1960s ?

what is the most 5 actors appearing in 1970s ?